In [1]:
import FinanceDataReader as fdr
import yfinance as yf
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from datetime import datetime, timedelta

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'GLOBAL'
ID = 'root'
PW = 'root'

nasdaq = list(fdr.StockListing('NASDAQ')['Symbol'])
sp = list(fdr.StockListing('SP500')['Symbol'])
etc = ['NRGU', 'FNGU', 'TQQQ']
symbols = nasdaq + sp + etc

# db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
# cursor = db.cursor()

### 1. 1. 나스닥, SP500, 기타종목 최신화
print(">> 1. 나스닥, SP500, 기타종목 최신화")

for idx in range(0, len(symbols), 30):
  symbol_list = symbols[idx:idx+30]
  df = yf.download(symbol_list, start = '2021-08-01')
  time.sleep(0.1)
  swapDf = df.swaplevel(0, 1, axis=1)
  for symbol in symbol_list:
    symbolDf = swapDf[symbol].rename(columns={'Open': 'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}).drop(['Adj Close'], axis=1)
    symbolDf['symbol'] = symbol
    symbolDf = symbolDf[['symbol', 'open', 'high', 'low', 'close', 'volume']]
    try:
      db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8', pool_pre_ping=True)
      conn = db_connection.connect()
      symbolDf.to_sql(name='stocks_price', con=db_connection, if_exists='append', index=True, index_label="date")
      conn.close()
    except Exception as ex:
      print(ex, symbol)
      pass

# for symbol in symbols[:5]:
#   try:
#     df = yf.download('TQQQ',start = '2021-01-01')
#     df1 = df.swaplevel(0, 1, axis=1)
#     df = fdr.DataReader(symbol, '2021-07-01').rename(columns={'Open': 'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}).drop(['Change'], axis=1)
#     df['symbol'] = symbol
#     df = df[['symbol', 'open', 'high', 'low', 'close', 'volume']]

#     db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8', pool_pre_ping=True)
#     conn = db_connection.connect()
#     df.to_sql(name='stocks_price', con=db_connection, if_exists='append', index=True, index_label="date")
#     conn.close()
#   except Exception as ex:
#     print(ex, symbol)
#     pass


# df = fdr.DataReader('BTC/KRW', '2021-07-01').rename(columns={'Open': 'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}).drop(['Change'], axis=1)
# df['symbol'] = 'BTC/KRW'
# df = df[['symbol', 'open', 'high', 'low', 'close', 'volume']]
# 1.2. 기존 데이터 제거
# print("1.2. 기존 데이터 제거 (%s)" %(date))
# cursor.execute("DELETE FROM stocks_price WHERE date = " + date)
# db.commit()
# db.close()


4255it [00:05, 761.21it/s]
C:\Users\Jack\AppData\Local\Programs\Python\Python38-32\lib\site-packages\FinanceDataReader\wikipedia\listing.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Symbol'] = df['Symbol'].str.replace('\.', '')


>> 1. 나스닥, SP500, 기타종목 최신화
[*********************100%***********************]  30 of 30 completed
[*********************100%***********************]  30 of 30 completed
[*********************100%***********************]  30 of 30 completed
[*********************100%***********************]  30 of 30 completed
[*********************100%***********************]  30 of 30 completed
[*********************100%***********************]  30 of 30 completed
[*********************100%***********************]  30 of 30 completed
[*********************100%***********************]  30 of 30 completed
[*********************100%***********************]  30 of 30 completed
[*********************100%***********************]  30 of 30 completed
[*********************100%***********************]  30 of 30 completed
[*********************100%***********************]  30 of 30 completed
[*********************100%***********************]  30 of 30 completed
[*********************100%***********************]

In [ ]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from datetime import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'GLOBAL'
ID = 'root'
PW = 'root'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

### 2. 볼린저밴드 초기세팅
print(">> 2. 볼린저밴드 초기세팅")

# 2.0. 기존 데이터 제거
print("2.0. 기존 데이터 제거")
cursor.execute("TRUNCATE boll")
db.commit()

# 2.1. 티커리스트 조회
print("2.1. 티커리스트 조회")
cursor.execute("SELECT DISTINCT ticker FROM stocks_price")
tickers = list(map(lambda x: x[0], cursor.fetchall()))

# 2.2. 기간 설정
print("2.2. 기간 설정")
start = '20210601'
end = str(datetime.now().date()).replace('-', '')
print("기간: (%s - %s)" %(start, end))

# 2.3. 새로운 데이터 업데이트
print("2.3. 새로운 데이터 업데이트 (%s - %s)" %(start, end))
day = 20
for ticker in tickers:
  try:
    # Get ticker's price
    cursor.execute("SELECT * FROM stocks_price WHERE ticker = '" + ticker + "' ORDER BY date DESC")
    priceDf = pd.DataFrame(cursor.fetchall())
    priceDf = priceDf.rename(columns={0: 'date', 1:'ticker', 2:'open', 3:'high', 4:'low', 5:'close', 6:'volume'}).drop(['open', 'high', 'low', 'volume'], axis=1)
    # Set Dataframe
    priceDf['period'] = day
    priceDf['low'] = 0
    priceDf['medium'] = 0
    priceDf['high'] = 0
    priceDf['bandWidth'] = 0.0
    priceDf['position'] = 0.0
    # Set bandWidth, Bollinger
    for idx in range(0, len(priceDf)):
      if idx + day > len(priceDf) - 1:
        continue
      copyDf = priceDf[idx:idx+day].loc[:, 'close'].copy()
      avg = copyDf.mean()
      std = copyDf.std()
      
      lo = round(avg - std * 2)
      me = round(avg)
      hi = round(avg + std * 2)
      bw = (hi - lo) / me
      pos = (priceDf.iloc[idx]['close'] - lo) / (hi - lo)
      priceDf.at[idx, 'low'] = lo
      priceDf.at[idx, 'medium'] = me
      priceDf.at[idx, 'high'] = hi
      priceDf.at[idx, 'bandWidth'] = bw
      priceDf.at[idx, 'position'] = pos

    bollDf = priceDf.reset_index().set_index('date').drop(['index'], axis=1)
    
    db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8', pool_pre_ping=True)
    conn = db_connection.connect()
    bollDf.to_sql(name='boll', con=db_connection, if_exists='append', index=True, index_label="date")        
    conn.close()
  except Exception as ex1:
    print('ex1', ex1)
    pass

In [ ]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from datetime import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

### 3. MFI 초기세팅
print(">> 3. MFI 초기세팅")

# 3.0. 기존 데이터 제거
print("3.0. 기존 데이터 제거")
cursor.execute("TRUNCATE mfi")
db.commit()

# 3.1. 티커리스트 조회
print("3.1. 티커리스트 조회")
cursor.execute("SELECT DISTINCT ticker FROM stocks_price")
tickers = list(map(lambda x: x[0], cursor.fetchall()))

# 3.2. 기간 설정
print("3.2. 기간 설정")
start = '20210601'
end = str(datetime.now().date()).replace('-', '')
print("기간: (%s - %s)" %(start, end))

# 3.3. 새로운 데이터 업데이트
print("3.3. 새로운 데이터 업데이트 (%s - %s)" %(start, end))
day = 10
for ticker in tickers:
  try:
    # Get ticker's price
    cursor.execute("SELECT * FROM stocks_price WHERE ticker = '" + ticker + "' ORDER BY date DESC")
    priceDf = pd.DataFrame(cursor.fetchall())
    priceDf = priceDf.rename(columns={0: 'date', 1:'ticker', 2:'open', 3:'high', 4:'low', 5:'close', 6:'volume'})
    # Set Dataframe
    priceDf['period'] = day
    priceDf['tp'] = 0
    priceDf['mfi'] = 0
    priceDf['mfi_diff'] = 0
    # Set typical price
    for idx in range(0, len(priceDf)):
      row = priceDf.iloc[idx]
      priceDf.at[idx, 'tp'] = round((row['high'] + row['low'] + row['close']) / 3)
    # Set MFI
    for idx in range(0, len(priceDf)):
      if idx + day > len(priceDf) - 1:
        continue
      positiveRMF = 0
      negativeRMF = 0          
      for i in range(idx, idx + day):
        today = priceDf.iloc[i]
        yesterday = priceDf.iloc[i + 1]
        if today['tp'] > yesterday['tp']:
          positiveRMF += today['tp'] * today['volume']
        elif today['tp'] < yesterday['tp']:
          negativeRMF += today['tp'] * today['volume']
      MFI = round(positiveRMF / (positiveRMF + negativeRMF) * 100)
      priceDf.at[idx, 'mfi'] = MFI
      if idx > 0:
        priceDf.at[idx - 1, 'mfi_diff'] = priceDf.iloc[idx - 1]['mfi'] - MFI
    mfiDf = priceDf.reset_index().set_index('date').drop(['index', 'open', 'high', 'low', 'close', 'volume'], axis=1)
    db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8', pool_pre_ping=True)
    conn = db_connection.connect()
    mfiDf.to_sql(name='mfi', con=db_connection, if_exists='append', index=True, index_label="date")        
    conn.close()
  except Exception as ex1:
    print('ex1', ex1, ticker)
    pass
db.close()